In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
pd.set_option('display.float_format', '{:.2f}'.format)

## Зареждане и поглед върху данните, с които ще работим

In [ ]:
dataset = pd.read_csv('./data/bg-car-offers.csv')

dataset.shape

Файлът с обяви за продажба на автомобили съдържа 99 693 записа (обяви) и 26 колони с данни за всяка обява. Нека разгледаме няколко записа от него:

In [ ]:
dataset.sample(10)

Виждаме, че файлът съдържа информация, която бихме очаквали да намерим, когато търсим да закупим автомобил: марката, модела, датата на производство, типа на двигателя и трансмисията, цвета, наличните екстри и оборудване, и, разбира се, цената на автомобила. Разполагаме също и с данни, които не описват предлагания автомобил, а са данни за самата обява като заглавие и брой преглеждания на обявата. Тези данни може да са полезни за анализа, но няма да ги включваме в модела за предсказване на цената на автомобила, тъй като искаме да предскажем цената на автомобила единствено на базата на неговите характеристики.

Поради факта, че тези данни са попълвани от потребители в интернет, на доста места се наблюдават непопълнени данни, които ще трябва да обработим преди да започнем да обучаваме какъвто и да е модел за предсказване на цената на автомобила.

Нека разгледаме типа на данните във файла:

In [ ]:
dataset.dtypes

Виждаме, че имаме данни от тип `object` (текстови данни), `float64` (дробни числа) и `int64` (цели числа).

Числовите колони са:
- `EngineSize` - обем на двигателя
- `Horsepower` - мощност на двигателя
- `Mileage` - пробег на автомобила
- `MilesPerFullCharge (EV)` - пробег на електрически автомобили с едно зареждане
- `BatteryCapacity (EV)` - капацитет на батерията на електрически автомобили
- `ViewCount` - брой преглеждания на обявата

Текстовите колони са:
- `Make` - марка на автомобила
- `Model` - модел на автомобила
- `BodyType` - тип на купето
- `ManufactureDate` - дата на производство
- `FuelType` - тип на горивото
- `Transmission` - тип на трансмисията
- `EuroStandard` - евростандарт на автомобила
- `Color` - цвят на автомобила
- `VIN` - VIN номер на автомобила
- `Price` - цена на автомобила (Интересно е, че тази колона не е от числов тип - трябва да разберем защо)
- `SafetyFeatures` - налични екстри за сигурност
- `ComfortFeatures` - налични екстри за комфорт
- `OtherFeatures` - други налични екстри
- `ExteriorFeatures` - външни екстри
- `InteriorFeatures` - вътрешни екстри
- `SpecialisedFeatures` - специализирани екстри
- `Region` - област, в която се намира автомобилът
- `City` - населено място, в което се намира автомобилът
- `OfferTitle` - заглавие на обявата

In [ ]:
dataset.describe(include='all')

## Price - целевата променлива

Нека започнем подробния анализ с нашата целева променлива - обявената цена на автомобила `Price`.
Да разгледаме разпределението на цените на автомобилите:

In [ ]:
prices = dataset['Price']

prices.value_counts(dropna=False)

Оказва се, че в доста от обявите няма определена точна цена, а вместо това е посочено `"При запитване"`. Това е и причината тази колона да не е от числов тип. Ще трябва да обработим тези записи преди да можем да използваме тази колона за обучение на модела. Да видим дали няма и други текстови стойности в тази колона:

In [ ]:
prices[prices.str.contains('[a-zA-Z]')].value_counts()

Имаме 8 обяви с цена, обявена в долари. Ще трябва да преобразуваме тези цени в левове, за да можем да ги използваме за обучение на модела.

Да видим как изглежда разпределението на цените на автомобилите:

In [ ]:
numeric_prices = prices[~prices.str.contains('[a-zA-Zа-яА-Я]')].astype(float).round()

numeric_prices.describe()

Веднага се забелязват няколко неща:
- най-ниската цена на автомобил е 11 лева, което е изключително ниска стойност и най-вероятно е въведена за привличане на внимание към обявата, т.е. бихме могли да я считаме за аутлайър и да не взимаме подобни цени под внимание, тъй като могат да повлият негативно на точността на модела
- най-високата цена на автомобил е 19 558 298 лева, което също е много висока стойност и може да бъде аутлайър
- 75% от автомобилите са на цена до 33 000 лева, едва 25% са на цена над тази стойност - доста по-големите цени са рядкост и може да се окаже, че са аутлайъри, които да не вземаме под внимание

Да видим как изглежда разпределението на цените на автомобилите графично:

In [ ]:
plt.figure(figsize=(12, 6))
plt.title('Разпределение на цените на автомобилите')
plt.xlabel('Цена (лв.)')
plt.ylabel('Брой обяви')
sns.histplot(numeric_prices, bins=100, kde=True)

Нека премахнем автомобилите с цена под 500 лева и над 150 000 лева, за да можем да видим по-детайлно разпределението на цените:

In [ ]:
numeric_prices_no_outliers = numeric_prices[(numeric_prices >= 500) & (numeric_prices <= 150000)]

plt.figure(figsize=(12, 6))
plt.title('Разпределение на цените на автомобилите без аутлайъри')
plt.xlabel('Цена (лв.)')
plt.ylabel('Брой обяви')
sns.histplot(numeric_prices_no_outliers, kde=True)

Графиката потвърждава видяното в таблицата по-горе - мнозинството от автомобилите са на цена до 33 000 лева, а по-големите цени са рядкост и можем да ги изключим от набора за обучение на модела, тъй като те ще изкривят значително предсказанията му.

In [ ]:
plt.figure(figsize=(12, 6))
plt.title('Разпределение на цените на автомобилите без аутлайъри')
plt.ylabel('Цена (лв.)')

sns.boxplot(numeric_prices_no_outliers)

Да видим колко празни стойности имаме в колоната `Price`:

In [ ]:
prices.isna().sum()

За щастие, нямаме празни стойности в тази колона.

## Марка, модел, тип на купето

Да разгледаме как са разпределени марките и моделите на автомобилите в нашия набор от данни:

In [ ]:
make_model_body = ['Make', 'Model', 'BodyType']

dataset[make_model_body].describe(include='all')

Имаме данни за 112 марки и 1403 модела автомобили, като най-често срещаният производител е Mercedes-Benz, а най-често срещаният модел е Golf (Volkswagen). Типовете купета са 11 на брой, като най-популярен е джипът.

Ще бъде интересно да разгледаме дяловете на различните марки и модели в нашия набор от данни.

In [ ]:
make_value_counts = dataset['Make'].value_counts()

plt.figure(figsize=(18, 6))

plt.subplot(1, 2, 1)
plt.title('Дял на обявите по марка')
plt.pie(make_value_counts, labels=make_value_counts.index, autopct='%1.1f%%')

plt.subplot(1, 2, 2)
plt.title('Брой на обявите за първите 15 марки автомобили')
plt.ylabel('Марка')
plt.xlabel('Брой обяви')
sns.barplot(x=make_value_counts.head(15).values, y=dataset['Make'].value_counts().head(15).index, orient='h')

За българските шофьори едва ли ще бъде изненада, че най-популярните марки автомобили у нас са немските производители на премиум автомобили - а именно Mercedes-Benz, BMW и Audi. Делът на тези 3 марки в нашия набор от данни е над 30%, което е впечатляващо. Близко след тях е друг немски производител - Volkswagen, който също е много популярен у нас. Останалите марки се задоволявят с доста по-малък дял от пазара, като те си поделят по около 1-2% от обявите.

Да видим как стоят нещата при моделите:

In [ ]:
model_value_counts = dataset['Model'].value_counts()

plt.title('Брой обяви за първите 15 модели автомобили')
plt.ylabel('Модел')
plt.xlabel('Брой обяви')
sns.barplot(x=model_value_counts.head(15).values, y=dataset['Model'].value_counts().head(15).index, orient='h')

Тук ситуацията изглежда по-балансирана - най-популярен е Golf на Volkswagen, но не доминира съществено над останалите модели. Сред първите 15 модела се нареждат предимно модели на немските производители, но виждаме и такива на японските производители Toyota и Honda, както и на чешкия Skoda.

Нека проверим как се разпределят типовете купета:

In [ ]:
body_type_value_counts = dataset['BodyType'].value_counts()

plt.figure(figsize=(12, 6))
plt.title('Дял на обявите по вид купе')
plt.pie(body_type_value_counts, labels=body_type_value_counts.index, autopct='%1.1f%%')

Пазарът е относително равномерно разпределен между различните типове купета, като най-голям дял имат джиповете и хечбековете - общо почти половината от обявите са за тези два типа купета. След тях са седаните и комбитата, които също притежават значителен дял от пазара.

Да проверим дали имаме празни стойности в колоните `Make`,`Model` и `BodyType`:

In [ ]:
dataset[make_model_body].isna().sum()

## Дата на производство

Нека проверим разпределението на автомобилите по тяхната година на производство:

In [ ]:
dataset['ManufactureDate'].describe(include='all')

In [ ]:
years_of_manufacture = dataset['ManufactureDate'].dropna().str.split('/').str[1].astype(int)

years_of_manufacture.describe()

In [ ]:
plt.title('Разпределение на автомобилите по година на производство')
plt.xlabel('Година на производство')
plt.ylabel('Брой обяви')

sns.histplot(years_of_manufacture, kde=True)

От таблицата и графиката е видно, че почти всички автомобили са произведени след 1990г., като най-голям брой от тях са произведени през новото хилядолетие. Най-старият автомобил в нашия набор от данни е произведен през 1931г. - по всяка вероятност това е класически автомобил, чиято цена едва ли ще бъде ниска, както би се очаквало от автомобил на голяма възраст. Нека го разгледаме:

In [ ]:
boolean_mask = (years_of_manufacture == 1931)
boolean_mask.index = dataset.index[years_of_manufacture.index]

dataset.dropna(subset='ManufactureDate').loc[boolean_mask]

Всъщност изглежда, че тази обява съдържа грешна информация, тъй като моделът е записан като Mustang, което не е възможно, тъй като моделът Мustang влиза в прозиводство едва през 1964г. Тази и подобни обяви ще трябва да бъдат премахнати от набора от данни, тъй като те могат да изкривят резултатите на модела.

Сега нека проверим как годината на производство влияе на цената на един автомобил: 

In [ ]:
# Ensure numeric_prices_no_outliers and years_of_manufacture have the same length
valid_indices = numeric_prices_no_outliers.index.intersection(years_of_manufacture.index)

# Filter dataset to include only valid rows
filtered_dataset = dataset.loc[valid_indices]
filtered_prices = numeric_prices_no_outliers.loc[valid_indices]
filtered_years = years_of_manufacture.loc[valid_indices]

plt.figure(figsize=(6, 12))
plt.title('Цена на автомобилите спрямо годината на производство')
plt.xlabel('Цена (лв.)')
plt.ylabel('Година на производство')
sns.boxplot(x=filtered_prices, y=filtered_years, orient='h')

Наблюдават се няколко интересни тенденции:
- 1951 - 1998г. - Средната цена на автомобилите по-скоро намалява с годините, с изключения в някои години. Това може би се дължи на факта, че автомобилите произведени в началото на този период са придобили колекционерска стойност, тъй като се срещат все по-рядко поради тяхната възраст, а тези от по-късния период все още се срещат често по пътищата и възрастта им по-скоро влияе негативно на цената им.
- 1998 - 2024г. - В този период средната цена на автомобилите нараства с годините, като от 2010г. нататък тази тенденция става все по-ясно изразена.

## За двигатели

Да разгледаме наличниите данни за двигателите в автомобилите:

In [ ]:
dataset.columns

Двигателите на автомобилите се описват от следните колони:
- `EngineSize` - обем на двигателя в кубични сантиметри
- `FuelType` - тип на горивото
- `Horsepower` - мощност на двигателя в конски сили
- `EuroStandard` - евро категория на двигателя

In [ ]:
engine_columns = ['EngineSize', 'FuelType', 'Horsepower', 'EuroStandard']

dataset[engine_columns].describe(include='all')

In [ ]:
valid_indices = numeric_prices_no_outliers.index.intersection(dataset['EngineSize'].dropna().index)
filtered_dataset = dataset.loc[valid_indices]
filtered_prices = numeric_prices_no_outliers.loc[valid_indices]

plt.title('Цена на автомобилите спрямо обема на двигателя')
ax = sns.regplot(data=filtered_dataset, x='EngineSize', y=filtered_prices)
ax.set(xlabel='Обем на двигателя (куб. см.)', ylabel='Цена (лв.)')

Обем на двигателя над 10 000 куб. см. изглежда като аномалия и не е възможно да има толкова много автомобили с толкова големи двигатели. Нека разгледаме тези записи:

In [ ]:
dataset[dataset['EngineSize'] > 10000].sample(15)

Изглежда, че тези записи съдържат грешни данни - при попълването на обема е добавена допълнителна нула. Тези записи ще трябва да бъдат поправени преди да ги използваме за обучение.

Да видим отново графиката като премахнем аномалиите:

In [ ]:
filtered_dataset = filtered_dataset[filtered_dataset['EngineSize'] < 10000]
filtered_prices = numeric_prices_no_outliers.loc[filtered_dataset.index]

plt.title('Цена на автомобилите спрямо обема на двигателя без аутлайъри')
ax = sns.regplot(data=filtered_dataset, x='EngineSize', y=filtered_prices, line_kws={'color': 'red'})
ax.set(xlabel='Обем на двигателя (куб. см.)', ylabel='Цена (лв.)')

Едва ли е изненадващо, че по-големият обем на двигателя води до по-висока цена на автомобила. По-големите двигатели обикновено са и по-мощни, което също води до по-висока цена.

In [ ]:
filtered_prices = numeric_prices_no_outliers.loc[filtered_dataset.index]

plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.title('Мощност на двигателя спрямо обема му')
ax = sns.regplot(data=filtered_dataset, x='EngineSize', y='Horsepower', line_kws={'color': 'red'})
ax.set(xlabel='Обем на двигателя (куб. см.)', ylabel='Мощност на двигателя (к.с.)')

plt.subplot(1, 2, 2)
plt.title('Цена на автомобила спрямо мощността му')
ax = sns.regplot(data=filtered_dataset, x='Horsepower', y=filtered_prices, line_kws={'color': 'red'})
ax.set(xlabel='Мощност на двигателя (к.с.)', ylabel='Цена (лв.)')

При мощността на двигателя също намираме аътлайъри - автомобили с мощност над 1000 конски сили. Тези записи едва ли са грешни - тези стойности е напълно възможно да бъдат достигнати от силно модифицирани или спортни автомобили. Нека разгледаме тези записи:

In [ ]:
dataset[dataset['Horsepower'] > 1000].sample(15)

Всъщност виждаме, че тези стойности са грешни - по-голямата част от тях съответстват на обем на двигателя (1600, 1800, 2000 и т.н.). На някой места обемът на двигателя е записан като мощност, а на други обратното. Тези записи ще трябва да бъдат поправени преди да ги използваме за обучение.

Да погледнем отново графиката, като премахнем аутлайърите:

In [ ]:
filtered_dataset = filtered_dataset[filtered_dataset['Horsepower'] < 1000]
filtered_prices = numeric_prices_no_outliers.loc[filtered_dataset.index]

plt.title('Цена на автомобилите спрямо мощността на двигателя без аутлайъри')
ax = sns.regplot(data=filtered_dataset, x='Horsepower', y=filtered_prices, line_kws={'color': 'red'})
ax.set(xlabel='Мощност на двигателя (к.с.)', ylabel='Цена (лв.)')

Връзката е очевидна - по-мощните автомобили са по-скъпи.

Да видим как са разпределени различните типове гориво:

In [ ]:
fuel_type_value_counts = dataset['FuelType'].value_counts()

plt.figure(figsize=(12, 6))
plt.title('Дял на обявите по вид гориво')
plt.pie(fuel_type_value_counts, labels=fuel_type_value_counts.index, autopct='%1.1f%%')

В България най-популярни са дизеловите автомобили като те доминират пазара - над половината от автомобилите са дизелови. Бензиновите автомобили също са популярни. Електрическите и хибридните автомобили са много редки у нас, като техния дял е общо около 3%.

In [ ]:
filtered_dataset = dataset.copy()
filtered_prices = numeric_prices_no_outliers.index.intersection(filtered_dataset.index)

filtered_dataset = filtered_dataset.loc[filtered_prices]
filtered_prices = numeric_prices_no_outliers.loc[filtered_prices]

plt.title('Цени на автомобилите спрямо вида на горивото')
plt.xlabel('Вид на горивото')
plt.ylabel('Цена (лв.)')
sns.boxplot(data=filtered_dataset, x='FuelType', y=filtered_prices, hue='FuelType')

По отношение на цената разбираме, че дизеловите и бензиновите автомобили са на сродни цени. Малко по-скъпи от тях са хибридите, последвани от електрическите такива. Най-скъпи са Plug-in хибридите.

In [ ]:
euro_standard_value_counts = dataset['EuroStandard'].value_counts()

plt.figure(figsize=(12, 6))
plt.title('Дял на обявите по евро стандарт')
plt.pie(euro_standard_value_counts, labels=euro_standard_value_counts.index, autopct='%1.1f%%')

Интересно е, че над 1/3 от колите са с категория Евро 6 - най-високата еко категория. На второто място е Евро 4, а много близо зад нея е Евро 5. Останалите категории не са много предлагани, от една страна, защото автомобилите от тези категории са вече доста стари, но и защото тези категории са обвързани с по-високи данъци и такси, което ги прави по-малко привлекателни за покупка. 

In [ ]:
valid_indices = numeric_prices_no_outliers.index.intersection(dataset[engine_columns].dropna().index)

# Filter dataset to include only valid rows
filtered_dataset = dataset.loc[valid_indices]
filtered_prices = numeric_prices_no_outliers.loc[valid_indices]

plt.title('Цена на автомобилите спрямо евро стандарта на двигателя')
plt.xlabel('Евро стандарт')
plt.ylabel('Цена (лв.)')
sns.boxplot(data=filtered_dataset, x='EuroStandard', y=numeric_prices_no_outliers, hue='EuroStandard')

Наблюдения:
- `Евро 1, 2, 3` - Няма особена разлика в средната цена на автомобилите от тези категории, като те са доста по-евтини от останалите
- `Евро 4, 5` - Средната цена на автомобилите от тези категории е малко по-висока спрямо предходните категории 1, 2 и 3, но по-ниска от тези от категориите 6
- `Евро 6` - Ценовият диапазон е доста по-голям от този на останалите категории

## Данни за електрически автомобили

Имаме и няколко колони съдържащи информация само за електрически автомобили - колоните `MilesPerFullCharge (EV)` (пробег на електрически автомобили с едно зареждане) и `BatteryCapacity (EV)` (капацитет на батерията на електрически автомобили). Да разгледаме тези данни:

In [ ]:
ev_columns = ['MilesPerFullCharge (EV)', 'BatteryCapacity (EV)']

dataset[ev_columns].describe(include='all')

Веднага виждаме, че имаме много празни стойности в тези колони, тъй като, както вече видяхме, електрическите автомобили са слабо представени в нашия набор от данни. Да видим колко са те:

In [ ]:
dataset[ev_columns].isna().sum()

Отново, както и при по-горе разгледаните колони, се виждат стойности, които са много високи/ниски и вероятно са грешни данни. Нека разгледаме разпределението на стойностите в тези колони:

In [ ]:
miles_per_full_charge = dataset['MilesPerFullCharge (EV)'].dropna().astype(float)
battery_capacity = dataset['BatteryCapacity (EV)'].dropna().astype(float)

plt.figure(figsize=(12, 6))
plt.title('Разпределение на пробега на електрически автомобили с едно зареждане и капацитета на батерията')
sns.boxplot(data=pd.DataFrame({'MilesPerFullCharge (EV)': miles_per_full_charge, 'BatteryCapacity (EV)': battery_capacity}))

Добра новина е, че тук аутлайрите изглежда са доста малко на брой. Нека все пак разгледаме някои:

In [ ]:
dataset[dataset['MilesPerFullCharge (EV)'] > 1000].head()

При тези автомобили е трудно да се определи от какво е предизвикана грешката, но със сигурност тези стойности са грешни. Понеже тези записи са едва 2 на брой, бихме могли просто да ги премахнем от набора от данни.

In [ ]:
dataset[dataset['BatteryCapacity (EV)'] > 150].sample(15)

Отново, тези стойности изглеждат като грешни данни. Бихме могли да ги заменим със средната стойност на колоната за съответния модел.

Сега нека разгледаме така важната връзка между пробега с едно зареждане и капацитета на батерията, и цената на автомобила:

In [ ]:
valid_indices = numeric_prices_no_outliers.index.intersection(dataset[ev_columns].dropna().index)

filtered_dataset = dataset.loc[valid_indices]
filtered_prices = numeric_prices_no_outliers.loc[valid_indices]

plt.ylim(0, 150000)
plt.xlim(0, 1000)
plt.title('Цена на автомобилите спрямо пробега с едно зареждане')
ax = sns.regplot(data=filtered_dataset, x='MilesPerFullCharge (EV)', y=filtered_prices, line_kws={'color': 'red'})
ax.set(xlabel='Пробег с едно зареждане (км.)', ylabel='Цена (лв.)')

Не виждаме изненади - пробегът с едно зареждане на електрическия автомобил е може би първото нещо, от което се интересува всеки потенциален купувач. Това е отразено и в търсената цена. Но на графиката ясно се открояват две групи наблюдения:
- автомобили с пробег до около 100км и цена, която варира в целия диапазон - Защо цената на автомобил с толкова малък пробег е толкова висока, обратно на тенденцията, която наблюдаваме?
- автомобили с вариращ пробег, при които видно цената се покачва с увеличаването на пробега

Една хипотеза, която бихме могли да проверим е, че този клъстър от скъпи автомобили с нисък пробег са всъщност хибридни автомобили, които комбинират електрическата енергия с тази, произведена от ДВГ (Двигател с вътрешно горене) и затова пробегът им на ток е нисък. Нека проверим тази хипотеза:

In [ ]:
plt.ylim(0, 150000)
plt.xlim(0, 1000)

plt.title('Цена на автомобилите спрямо пробега с едно зареждане')
plt.xlabel('Пробег с едно зареждане (км.)')
plt.ylabel('Цена (лв.)')
sns.scatterplot(data=filtered_dataset, x='MilesPerFullCharge (EV)', y=filtered_prices, hue='FuelType')

Изглежда, че тази хипотеза е вярна - клъстерът е съставен изключително от Plug-in хибриди, чиято цена варира драстично в зависимост от модела и марката.

## Пробег

Следва да разгледаме изключително важна характеристика за всеки употребяван автомобил - неговият пробег. Всеки шофьор знае, че големият пробег намалява цената на автомобила драстично. Нека проверим как всъщност пробегът влияе на цената на автомобила:

In [ ]:
dataset['Mileage'].describe(include='all')

Средният пробег на употребяван автомобил в България е ок. 180 000 км. Отново срещаме подозрително високи/ниски стойности, които ще трябва да премахнем. 

In [ ]:
miles = dataset['Mileage'].dropna().astype(float)
miles.drop(miles[miles > 500000].index, inplace=True)

sns.displot(miles, kde=True)

Повечето автомобили са с пробег между 150 000 и 250 000 км, като стойностите над 400 000 км са много редки и ще ги считаме за аутлайъри.

In [ ]:
plt.title('Разпределение на пробега на автомобилите')
plt.ylabel('Пробег (км.)')
sns.boxplot(miles)

In [ ]:
valid_indices = numeric_prices_no_outliers.index.intersection(miles.index)

filtered_dataset = dataset.loc[valid_indices]
filtered_prices = numeric_prices_no_outliers.loc[valid_indices]

plt.ylim(0, 150000)
plt.title('Цена на автомобилите спрямо пробега')
plt.xlabel('Пробег (км.)')
plt.ylabel('Цена (лв.)')
sns.regplot(data=filtered_dataset, x='Mileage', y=filtered_prices, line_kws={'color': 'red'})

До около 100 000 км. наблюдаваме автомобили с цени, разпръснати в целия ценови диапазон - явно пробегът не оказва достатъчно голямо влияние при такива автомобили. След тази граница, обаче, цената започва видимо да намалява с увеличаването на пробега.

## Данни за местоположението

Колоните описващи местоположението на автомобила също представляват интерес за нас. Икономическота положение може да варира драстично в различните области на България и това може да окаже влияние на паричната стойност на автомобилите, които хората употребяват в тях. Да разгледаме тези колони:

In [ ]:
location_columns = ['Region', 'City']

dataset[location_columns].describe(include='all')

Имаме автомобили от всички области в България, но има и още една стойност, която не знаем какво представлява - областите в България са само 28. Да погледнем стойностите в тази колона по-подробно:

In [ ]:
dataset['Region'].value_counts()

Оказва се, че 29-тата област всъщност показва автомобили от чужбина. За всеки случай би било добре да премахнем такива записи от данните, тъй като в дадената цена може да присъстват разходи за внасяне на автомобила в страната, които не произлизат от самия автомобил. Без изненада най-много автомобили се продават в столицата - София, следвана от Пловдив и Варна. Да видим как са разпределени областите по брой обяви:

In [ ]:
plt.figure(figsize=(12, 6))
plt.title('Дял на обявите по области')
plt.pie(dataset['Region'].value_counts(), labels=dataset['Region'].value_counts().index, autopct='%1.1f%%')

Над половината от обявите са в трите най-големи области в България - София, Пловдив и Варна. Да видим дали в областите, където се намират най-големите градове, цените на автомобилите са по-високи:

In [ ]:
valid_indices = numeric_prices_no_outliers.index.intersection(dataset[location_columns].dropna().index)

filtered_dataset = dataset.loc[valid_indices]
filtered_prices = numeric_prices_no_outliers.loc[valid_indices]

plt.title('Цена на автомобилите спрямо областта')
plt.xlabel('Цена (лв.)')
plt.ylabel('Област')
sns.boxplot(data=filtered_dataset, x=filtered_prices, y='Region')

Единствената по-голяма разлика в цените виждаме при автомобилите от чужбина. Този факт потвърждава гореизложената хипотеза, че цената на автомобилите от чужбина може да е по-висока поради разходите за внасяне на автомобила и свързани разходи.

## Екстри

Нека видим и наличната информация за екстрите, с които разполага всеки автомобил. Тези данни със сигурност имат голямо знаяение за цената на автомобила и ще бъде от голяма полза да ги разгледаме подробно и да ги преработим по най-добрия начин:

In [ ]:
dataset.columns

In [ ]:
extras_columns = ['SafetyFeatures', 'ComfortFeatures', 'OtherFeatures', 'ExteriorFeatures', 'InteriorFeatures', 'SecurityFeatures', 'SpecialisedFeatures']

dataset[extras_columns].sample(15)

Данните не са представени по най-добрия начин за обработка и анализ. Екстрите са групирани в няколко категории, като всяка категория съдържа списък от екстри, разделени със запетаи. Това не е подходящо за анализ, както и за обучение на модел, тъй като всеки списък от екстри ще бъде третиран като една стойност, като това не само драстично увеличава възможните стойности/пермутации (кардиналността) на колоната, но и ще повлияе негативно на качеството на модела, тъй като не всички екстри оказват еднакво влияние върху цената на автомобила, и това ще попречи на модела да научи тези връзки, когато екстри с различно влияние са групирани в една стойност.
Затова ще бъде добре при обработката на данните да отделим всяка екстра в отделна колона, като стойността на всяка колона ще бъде булева - дали даден автомобил разполага с тази екстра или не. Това ще ни позволи да анализираме влиянието на всяка екстра върху цената на автомобила и ще подготви данните за обучение на модела.

In [ ]:
exploded_features = dataset[extras_columns].copy()

exploded_features = exploded_features.fillna('')

for column in exploded_features.columns:
    exploded_features = exploded_features.join(exploded_features[column].str.get_dummies(sep=', ').add_prefix(column + '_'))

exploded_features.sample(15)

In [ ]:
dataset_with_exploded_features = dataset.join(exploded_features.drop(columns=extras_columns))

dataset_with_exploded_features.drop(columns=extras_columns, inplace=True)

dataset_with_exploded_features.sample(15)

Може би ще бъде най-подходящо да разгледаме връзката между броя на екстрите във всяка категория и цената на автомобила:

In [ ]:
dataset_with_total_features = dataset_with_exploded_features.copy()

dataset_with_total_features['TotalSafetyFeatures'] = dataset_with_exploded_features.filter(like='SafetyFeatures').sum(axis=1)
dataset_with_total_features['TotalComfortFeatures'] = dataset_with_exploded_features.filter(like='ComfortFeatures').sum(axis=1)
dataset_with_total_features['TotalOtherFeatures'] = dataset_with_exploded_features.filter(like='OtherFeatures').sum(axis=1)
dataset_with_total_features['TotalExteriorFeatures'] = dataset_with_exploded_features.filter(like='ExteriorFeatures').sum(axis=1)
dataset_with_total_features['TotalInteriorFeatures'] = dataset_with_exploded_features.filter(like='InteriorFeatures').sum(axis=1)
dataset_with_total_features['TotalSecurityFeatures'] = dataset_with_exploded_features.filter(like='SecurityFeatures').sum(axis=1)
dataset_with_total_features['TotalSpecialisedFeatures'] = dataset_with_exploded_features.filter(like='SpecialisedFeatures').sum(axis=1)

dataset_with_total_features.sample(15)

In [ ]:
total_features_columns = ['TotalSafetyFeatures', 'TotalComfortFeatures', 'TotalOtherFeatures', 'TotalExteriorFeatures', 'TotalInteriorFeatures', 'TotalSecurityFeatures', 'TotalSpecialisedFeatures']

valid_indices = numeric_prices_no_outliers.index.intersection(dataset_with_total_features.index)

filtered_dataset = dataset_with_total_features.loc[valid_indices]
filtered_prices = numeric_prices_no_outliers.loc[valid_indices]

plt.figure(figsize=(16, 12))
plt.suptitle('Цена на автомобилите спрямо броя на екстрите')

for column in total_features_columns:
    plt.subplot(3, 3, total_features_columns.index(column) + 1)
    ax = sns.regplot(data=filtered_dataset, x=f'{column}', y=filtered_prices, line_kws={'color': 'red'})
    ax.set(xlabel=f'Брой на екстрите в категория {column.removeprefix("Total").removesuffix('Features')}', ylabel='Цена (лв.)')

Всички графики показват, че по-голям брой на екстрите във всяка категория води до по-висока цена. Единствено при специализираните екстри това не е така. Нека разгледаме тези екстри по-подробно: 

In [ ]:
dataset_with_exploded_features.filter(like='SpecialisedFeatures').columns.values

Откриваме, че всъщност специализираните екстри не представляват отделни подобрения, а представляват специализирана версия на автомобила като: Линейка, Катафалка, Учебен, предназначен за хора с увреждания и др. Струва си да разгледаме как всяка от тези колони влияе на цената на автомобила:

In [ ]:
plt.subplots(2, 4, figsize=(16, 8))
plt.suptitle('Цена на автомобилите при наличие/отсъствие на специализирана екстра')

for column in dataset_with_exploded_features.filter(like='SpecialisedFeatures').columns:
    plt.subplot(2, 4, dataset_with_exploded_features.filter(like='SpecialisedFeatures').columns.get_loc(column) + 1)
    sns.boxplot(data=filtered_dataset, x=column, y=filtered_prices)
    plt.xlabel(column.removeprefix('SpecialisedFeatures_'))
    plt.ylabel('Цена (лв.)')

За повечето колони наблюдаваме спад на цената, ако автомобилът е класифициран с някоя от тези категории. Това е разбираемо - такситата, линейките, катафалките и учебните автомобили обикновено имат много голям пробег поради интензивната им употреба и съответно са по-евтини. Единствено автомобилите с хомологация N1 са по-скъпи от останалите. При проверка какво всъщност означава автомобил да притежава тази категория откриваме, че автомобилите от клас N1 са товарни автомобили с маса до 3.5 тона (т.е. леки автомобили от категория B) и за тях се прилагат данъчни облекчения, тъй като често такива автомобили са фирмени и се използват за търговска дейност. Именно това ги прави по-скъпи от останалите.

## Цвят, VIN номер

Тези колони едва ли ще ни подскажат много за цената на автомобила, но все пак ще ги разгледаме:

In [ ]:
dataset[['Color', 'VIN']].describe()

Да видим все пак дали цвета оказва ефект върху цената:

In [ ]:
plt.figure(figsize=(12, 7))
plt.title('Цена на автомобилите спрямо цвета')
plt.xlabel('Цена (лв.)')
plt.ylabel('Цвят')
sns.boxplot(data=filtered_dataset, x=numeric_prices_no_outliers, y='Color')

Няма ясна връзка между цвета на автомобила и цената му. Единствено някои цветове като зелен, розов, резидав и банан не са много предпочитани и цената им е по-ниска.

## Корелация на колоните с цената на автомобила

### Корелация на числовите колони с цената

In [ ]:
correlation_price = filtered_dataset.select_dtypes(include=np.number).loc[filtered_prices.index].corrwith(filtered_prices)

plt.figure(figsize=(6, 21))
plt.title('Корелация на числовите колони с цената на автомобила')
plt.xlabel('Корелация')
plt.ylabel('Колона')

sns.barplot(correlation_price.sort_values(ascending=False), orient='h')

Видно е, че повечето колони имат положителна корелация с цената - по-голямата стойност в колоната допринася за по-висока цена, както вече установихме при прегледа на числовите колони. Откриваме и няколко колони с отрицателна корелация - по-голямата стойност в тях води до по-ниска цена. Това са колони като пробега (`Mileage`)

### Корелация на числовите колони помежду им

In [ ]:
correlation_matrix = filtered_dataset.select_dtypes(include=np.number).loc[filtered_prices.index].corr()

plt.figure(figsize=(22, 20))
plt.title('Корелационна матрица на числовите колони', fontsize=20)

sns.heatmap(correlation_matrix, fmt='.2f', cmap='coolwarm')

Открояват се няколко колони с много силна корелация помежду си:
- `BatterSize (EV)` и `MilesPerFullCharge (EV)` - по-голямата батерия води до по-голям пробег, показват едно и също нещо
- `ComfortFeatures_TV` и `ComfortFeatures_DVD` - телевизорът и DVD-то винаги се срещат заедно
- `ComfortFeatures_USB`, `ComfortFeatures_audio/video` с `ComfortFeatures_IN\AUX изводи` - всички тези колони показват наличие на мултимедия в автомобила
- `ComfortFeatures_Климатик` и `ComfortFeatures_Климатроник` - силна отрицателна корелация, при наличие на климатроник, климатикът липсва и обратното
- `OtherFeatures_С регистрация` и `OtherFeatures_Нов внос` - силна отрицателна корелация, при нов внос автомобилът няма регистрация и обратно
- `ExteriorFeatures_2(3) врати` и `ExteriorFeatures_4(5) врати` - силна отрицателна корелация, автомобилите с 2(3) врати нямат 4(5) врати и обратно